## Dads Club Excel Spreadsheet Formatter

Load the sheet and then drop the columns

### Imports

In [1]:
import re
import warnings
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

from keras.models import Sequential, load_model
from keras.layers import Dense, LSTM, Embedding, Dropout
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
warnings.filterwarnings('ignore')


Using TensorFlow backend.


### Loading in Dataset

In [2]:
from pandas import ExcelWriter
from pandas import ExcelFile

pd.set_option('display.max_colwidth', -1)

#df = pd.read_excel('./transactions.xlsx', sheetname='transactions-2020-03-01-2020-03')

df = pd.read_csv('./trans.csv')

print("Column headings:")
list(df.columns)


Column headings:


['Date',
 'Time',
 'Time Zone',
 'Gross Sales',
 'Discounts',
 'Net Sales',
 'Gift Card Sales',
 'Tax',
 'Tip',
 'Partial Refunds',
 'Total Collected',
 'Source',
 'Card',
 'Card Entry Methods',
 'Cash',
 'Square Gift Card',
 'Other Tender',
 'Other Tender Type',
 'Other Tender Note',
 'Fees',
 'Net Total',
 'Transaction ID',
 'Payment ID',
 'Card Brand',
 'PAN Suffix',
 'Device Name',
 'Staff Name',
 'Staff ID',
 'Details',
 'Description',
 'Event Type',
 'Location',
 'Dining Option',
 'Customer ID',
 'Customer Name',
 'Customer Reference ID',
 'Device Nickname',
 'Deposit ID',
 'Deposit Date',
 'Deposit Details',
 'Fee Percentage Rate',
 'Fee Fixed Rate',
 'Refund Reason',
 'Discount Name']

In [3]:
df.head()


,Date,Time,Time Zone,Gross Sales,Discounts,Net Sales,Gift Card Sales,Tax,Tip,Partial Refunds,...,Customer Name,Customer Reference ID,Device Nickname,Deposit ID,Deposit Date,Deposit Details,Fee Percentage Rate,Fee Fixed Rate,Refund Reason,Discount Name
0,03/09/20,19:08:30,Arizona,$140.00,$0.00,$140.00,$0.00,$5.60,$0.00,$0.00,...,Alexander de Soler,11ea627408311329a1470cc47a2aeaec,NaN,NaN,NaN,NaN,2.9,$0.30,NaN,NaN
1,03/09/20,15:42:38,Arizona,$40.00,$0.00,$40.00,$0.00,$1.60,$0.00,$0.00,...,Lisa Counters,11ea6257465e8b41a1470cc47a2aeaec,NaN,NaN,NaN,NaN,2.9,$0.30,NaN,NaN
2,03/09/20,09:54:45,Arizona,$20.00,$0.00,$20.00,$0.00,$0.80,$0.00,$0.00,...,Angela Wallace,11ea6226acdd142fa1470cc47a2aeaec,NaN,NaN,NaN,NaN,2.9,$0.30,NaN,NaN
3,03/08/20,19:05:46,Arizona,$60.00,$0.00,$60.00,$0.00,$2.40,$0.00,$0.00,...,sean warfield,11ea61aa7cca9539a1470cc47a2aeaec,NaN,NaN,NaN,NaN,2.9,$0.30,NaN,NaN
4,03/07/20,19:25:53,Arizona,$20.00,$0.00,$20.00,$0.00,$0.80,$0.00,$0.00,...,Jesse Young,NaN,NaN,3ZD337ZBB1NR8YN8EEBY12E74TQS,03/08/20,https://squareup.com/dashboard/sales/deposits/3ZD337ZBB1NR8YN8EEBY12E74TQS/by-unit/B439B6GCX8P8S,2.6,$0.10,NaN,NaN


In [4]:

df=df.drop([
 'Time',
 'Time Zone',
 'Discounts',
 'Net Sales',
 'Gift Card Sales',
 'Tax',
 'Tip',
 'Partial Refunds',
 'Total Collected',
 'Source',
 'Card',
 'Card Entry Methods',
 'Cash',
 'Square Gift Card',
 'Other Tender',
 'Other Tender Type',
 'Other Tender Note',
 'Fees',
 'Net Total',
 'Card Brand',
 'PAN Suffix',
 'Device Name',
 'Staff Name',
 'Staff ID',
 'Description',
 'Event Type',
 'Location',
 'Dining Option',
 'Customer ID',
 'Customer Reference ID',
 'Device Nickname',
 'Deposit ID',
 'Deposit Date',
 'Deposit Details',
 'Fee Percentage Rate',
 'Fee Fixed Rate',
 'Refund Reason',
 'Discount Name'], axis=1)
list(df.columns)

['Date',
 'Gross Sales',
 'Transaction ID',
 'Payment ID',
 'Details',
 'Customer Name']

In [5]:

df.head()

,Date,Gross Sales,Transaction ID,Payment ID,Details,Customer Name
0,03/09/20,$140.00,vNmq8J7v88BusFao4gCieuyvLi4F,R4gkcgf1w6EHQwkxqKD3lcQfCSeZY,https://squareup.com/dashboard/sales/transactions/vNmq8J7v88BusFao4gCieuyvLi4F/by-unit/B439B6GCX8P8S,Alexander de Soler
1,03/09/20,$40.00,laB3wsFkDy7q2hq0pR8ir4LxFg4F,vVLGUwwdfCAjH5N4DcnuYENCe2DZY,https://squareup.com/dashboard/sales/transactions/laB3wsFkDy7q2hq0pR8ir4LxFg4F/by-unit/B439B6GCX8P8S,Lisa Counters
2,03/09/20,$20.00,58plOjNpZQHqezrh6qcCyiruPf4F,l0D73ZxBnw8EpPrJtUDjoopkQgLZY,https://squareup.com/dashboard/sales/transactions/58plOjNpZQHqezrh6qcCyiruPf4F/by-unit/B439B6GCX8P8S,Angela Wallace
3,03/08/20,$60.00,TjSL4qL8yKy3iuu6VTXG4dX9Pi4F,XnQLpkoXD9Gv83AXnhNPVA2x9HfZY,https://squareup.com/dashboard/sales/transactions/TjSL4qL8yKy3iuu6VTXG4dX9Pi4F/by-unit/B439B6GCX8P8S,sean warfield
4,03/07/20,$20.00,uKJnYSFKY114Jj1K4cgYd6weV,RwaplJebbFjdAKLMpQZen3aXvaB,https://squareup.com/dashboard/sales/transactions/uKJnYSFKY114Jj1K4cgYd6weV/by-unit/B439B6GCX8P8S,Jesse Young


In [6]:

df['Transaction ID'] = df['Transaction ID'].apply(lambda x: 'https://squareup.com/dashboard/orders/overview/'+x) #transform text to lowercase
df.head()

,Date,Gross Sales,Transaction ID,Payment ID,Details,Customer Name
0,03/09/20,$140.00,https://squareup.com/dashboard/orders/overview/vNmq8J7v88BusFao4gCieuyvLi4F,R4gkcgf1w6EHQwkxqKD3lcQfCSeZY,https://squareup.com/dashboard/sales/transactions/vNmq8J7v88BusFao4gCieuyvLi4F/by-unit/B439B6GCX8P8S,Alexander de Soler
1,03/09/20,$40.00,https://squareup.com/dashboard/orders/overview/laB3wsFkDy7q2hq0pR8ir4LxFg4F,vVLGUwwdfCAjH5N4DcnuYENCe2DZY,https://squareup.com/dashboard/sales/transactions/laB3wsFkDy7q2hq0pR8ir4LxFg4F/by-unit/B439B6GCX8P8S,Lisa Counters
2,03/09/20,$20.00,https://squareup.com/dashboard/orders/overview/58plOjNpZQHqezrh6qcCyiruPf4F,l0D73ZxBnw8EpPrJtUDjoopkQgLZY,https://squareup.com/dashboard/sales/transactions/58plOjNpZQHqezrh6qcCyiruPf4F/by-unit/B439B6GCX8P8S,Angela Wallace
3,03/08/20,$60.00,https://squareup.com/dashboard/orders/overview/TjSL4qL8yKy3iuu6VTXG4dX9Pi4F,XnQLpkoXD9Gv83AXnhNPVA2x9HfZY,https://squareup.com/dashboard/sales/transactions/TjSL4qL8yKy3iuu6VTXG4dX9Pi4F/by-unit/B439B6GCX8P8S,sean warfield
4,03/07/20,$20.00,https://squareup.com/dashboard/orders/overview/uKJnYSFKY114Jj1K4cgYd6weV,RwaplJebbFjdAKLMpQZen3aXvaB,https://squareup.com/dashboard/sales/transactions/uKJnYSFKY114Jj1K4cgYd6weV/by-unit/B439B6GCX8P8S,Jesse Young


In [7]:
df.to_csv('transactions.csv', index=False)